# Analise de dados ENEM 2017

# 0.0 Imports

In [1]:
import inflection
import sqlite3
import zipfile
import pandas                   as pd
import numpy                    as np
import seaborn                  as sns
import plotly.express           as px
from sqlalchemy                 import create_engine
from matplotlib                 import gridspec
from matplotlib                 import pyplot as plt

from geopy.geocoders            import Nominatim
from geopy.extra.rate_limiter   import RateLimiter

## Funções

In [2]:
# Função para formatar notação científica
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Tamanho dos Graficos
sns.set(style = "darkgrid")
sns.set(rc={"figure.figsize":(15, 10)})
#pd.options.display.float_format = "{:,.2f}".format

def show_dimensions(data):
    print(f'Numbers of rows: { df.shape[0] }\n')
    print(f'Numbers of columns: { df.shape[1] }\n')
    return None

def show_dtypes(data):
    print(data.info())
    return None

# Obter coordenadas geográficas
def geo_data(data):
    """Essa Função busca a coordenadas geográficas a partir do dataset cidades e cria as colunas lat e long"""
    locator = Nominatim(user_agent="mygeocoder")
    geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

    df_cidades['geolocalizacao'] = df_cidades['Município'].apply(geocode)
    ## Filtrando lat e long da coluna
    df_cidades['lat']= df_cidades['geolocalizacao'].apply(lambda loc: loc.latitude if loc else None)
    df_cidades['long']= df_cidades['geolocalizacao'].apply(lambda loc: loc.longitude if loc else None)
       
    return data

def clean_code(data):
    """Esta Função Limpa o DataFrame
    Tipo de limpeza:
    remove os dados NA
    remove as notas zero """
    df = df.dropna()
    
    return data
    

##  Extração  dos dados

In [3]:
#MIcrodados Enem
colunas = ['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA','TP_ESCOLA', 
           'NO_MUNICIPIO_PROVA', 'SG_UF_PROVA','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT','NU_NOTA_REDACAO','Q006']

with zipfile.ZipFile('data/microdados_enem_2017.zip') as z:
    with z.open('DADOS/MICRODADOS_ENEM_2017.csv') as f:
        # Criar uma lista com estas colunas que serão usadas
        df = pd.read_csv(f,sep=';', encoding='ISO-8859-1', usecols=colunas, nrows=10000)
        
# carregar o Dataset cidades
colunas = ['Município','IDHMunicipal_2010', 'População_estimada_pessoas_2019']

df_cidades = pd.read_csv('data/cidades.csv', usecols=colunas)


In [5]:
geo_data(df_cidades)

,Município,População_estimada_pessoas_2019,IDHMunicipal_2010,geolocalizacao,lat,long
0,Adamantina,35068,"0,79","(Adamantina, Região Imediata de Adamantina-Luc...",-21.6867,-51.0763
1,Adolfo,3562,"0,73","(Adolfo, Região Imediata de São José do Rio Pr...",-21.2357,-49.6442
2,Aguaí,36305,"0,715","(Aguaí, Região Imediata de São João da Boa Vis...",-22.0592,-46.9794
3,Águas da Prata,8180,"0,781","(Águas da Prata, Região Imediata de São João d...",-21.9464,-46.7192
4,Águas de Lindóia,18705,"0,745","(Águas de Lindóia, Região Imediata de Amparo, ...",-22.4719,-46.6291
...,...,...,...,...,...,...
5565,Várzea,5500,"0,626","(Várzea, Região Geográfica Imediata de Patos, ...",-6.7711,-36.9912
5566,Venha-Ver,4177,"0,555","(Venha-Ver, Região Geográfica Imediata de Pau ...",-6.3205,-38.4899
5567,Vera Cruz,12481,"0,587","(Vera Cruz, Região Imediata de Marília, Região...",-22.2213,-49.8243
5568,Viçosa,1718,"0,592","(Viçosa, Região Geográfica Imediata de Viçosa,...",-20.7539,-42.8816


##  Transformação

In [8]:
#Renomear colunas para concatenar os datasets
df_cidades.columns=['NO_MUNICIPIO_PROVA',  'populacao', 'geolocalizacao', 'idh', 'lat', 'long',]

df = pd.merge(df, df_cidades, how='left', on='NO_MUNICIPIO_PROVA')

In [9]:
show_dimensions(df)

show_dtypes(df)

Numbers of rows: 11011

Numbers of columns: 20

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11011 entries, 0 to 11010
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NU_INSCRICAO        11011 non-null  int64  
 1   NU_ANO              11011 non-null  int64  
 2   TP_FAIXA_ETARIA     11011 non-null  int64  
 3   TP_SEXO             11011 non-null  object 
 4   TP_ESTADO_CIVIL     11011 non-null  int64  
 5   TP_COR_RACA         11011 non-null  int64  
 6   TP_ESCOLA           11011 non-null  int64  
 7   NO_MUNICIPIO_PROVA  11011 non-null  object 
 8   SG_UF_PROVA         11011 non-null  object 
 9   NU_NOTA_CN          7748 non-null   float64
 10  NU_NOTA_CH          8101 non-null   float64
 11  NU_NOTA_LC          8101 non-null   float64
 12  NU_NOTA_MT          7748 non-null   float64
 13  NU_NOTA_REDACAO     8101 non-null   float64
 14  Q006                11011 non-null  object 
 15  popul

In [12]:
#Renomear as colunas do dataset microdados
cols_old = ['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_ESCOLA', 'NO_MUNICIPIO_PROVA',
       'SG_UF_PROVA', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'NU_NOTA_REDACAO', 'Q006', 'populacao', 'geolocalizacao', 'idh', 'lat',
       'long']


#Função para renomear colunas
snakecase = lambda x: inflection.underscore(x)
cols_new = list( map( snakecase, cols_old) )
#Rename
df.columns = cols_new

In [13]:
df.columns = ['inscricao', 'ano', 'faixa_etaria', 'sexo',
       'estado_civil', 'cor_raca', 'tp_escola',
       'municipio', 'estado', 'ciencias_natureza', 'ciencias_humanas',
       'linguagens_codigos', 'matematica', 'redacao', 'renda_familiar', 'populacao', 'geolocalizacao','idh', 'lat',
       'long', ]

In [14]:
df.isna().sum()

inscricao                0
ano                      0
faixa_etaria             0
sexo                     0
estado_civil             0
cor_raca                 0
tp_escola                0
municipio                0
estado                   0
ciencias_natureza     3263
ciencias_humanas      2910
linguagens_codigos    2910
matematica            3263
redacao               2910
renda_familiar           0
populacao                0
geolocalizacao           0
idh                      0
lat                      0
long                     0
dtype: int64

In [52]:
#Exluindo os valores nulos
df = df.dropna()

df.drop(['geolocalizacao'], axis=1, inplace=True)

In [53]:
show_dimensions(df)

Numbers of rows: 7738

Numbers of columns: 21



In [54]:
# Cria nota média
df['nota_media'] = (df['ciencias_natureza']+
                    df['ciencias_humanas'] +
                    df['linguagens_codigos']+ 
                    df['matematica'] +
                    df['redacao']) / 5

#eliminando os estudantes que ficaram com nota 0 por qualquer outro motivo (e.g. eliminados)
df = df[df['nota_media'] != 0]

In [55]:
regiao = {
    "Norte":['AM', 'PA', 'RR', 'AC','RO','AP', 'TO'],
    "Sul":['PR', 'SC', 'RS'],
    "Centro_Oeste":['MT', 'MS', 'GO', 'DF'],
    "Nordeste":['BA', 'CE','MA', 'PB', 'PE', 'RN', 'SE', 'AL', 'PI'],
    "Sudeste":['SP', 'RJ', 'MG','ES']}

for r in regiao:
    df.loc[ df['estado'].isin(regiao[r]), 'regiao'] = r

In [82]:
#Mudar dtypes
df['idh']= df['idh'].str.replace(',','.').astype(float)

## Carregar Banco de Dados

In [83]:
df.dtypes


inscricao               int64
ano                     int64
faixa_etaria            int64
sexo                   object
estado_civil            int64
cor_raca                int64
tp_escola               int64
municipio              object
estado                 object
ciencias_natureza     float64
ciencias_humanas      float64
linguagens_codigos    float64
matematica            float64
redacao               float64
renda_familiar         object
populacao               int64
idh                   float64
lat                   float64
long                  float64
nota_media            float64
regiao                 object
dtype: object

In [86]:
query_create_table_microdados_enem =  """
    CREATE TABLE microdados_enem (
      inscricao               INTEGER,
      ano                     INTEGER,
      faixa_etaria            INTEGER,
      sexo                    TEXT,
      estado_civil            INTEGER,
      cor_raca                INTEGER,
      tp_escola               INTEGER,
      municipio               TEXT,
      estado                  TEXT,
      ciencias_natureza       REAL,
      ciencias_humanas        REAL,
      linguagens_codigos      REAL,
      matematica              REAL,
      redacao                 REAL,
      renda_familiar          TEXT,
      populacao               INTEGER,
      idh                     REAL,
      lat                     REAL,
      long                    REAL,
      nota_media              REAL,
      regiao                  TEXT
     )
"""

In [87]:
#Create table
conn = sqlite3.connect('db_enem.sqlite')
conn.execute(query_create_table_microdados_enem)
conn.commit()
conn.close()

In [88]:
conn = create_engine('sqlite:///db_enem.sqlite', echo=False)

#insert data
df.to_sql('microdados_enem', con=conn, if_exists='append', index=False)

7738

In [89]:
#Consultando o banco de dados
query ="""
    SELECT *  FROM microdados_enem
"""
pd.read_sql_query(query, conn)
df.shape

(7738, 21)

In [91]:
data= df.copy()

In [92]:
data.to_csv('data/enem.csv', index=False)

C:\Users\PC\SejaUmDataScientist\Analise_de_dados\dadosfera\tutorial-env\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
